#### Wednesday, December 20, 2023

[LangChain Retrieval QA with Instructor Embeddings & ChromaDB for PDFs](https://www.youtube.com/watch?v=cFCGUjc33aU)

https://colab.research.google.com/drive/17eByD88swEphf-1fvNOjf_C79k0h2DgF?usp=sharing

This kinda all works ... some confusion wih the db folder because this notebook uses the same folder name as the 'YT_Chroma_DB_Multi_doc_retriever_Langchain_Part1.ipynb' notebook. 


In [ ]:
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.0/782.0 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.8/248.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.4/922.4 kB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [1]:
!pip show langchain

Name: langchain
Version: 0.0.350
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: None
Author-email: None
License: MIT
Location: /usr/local/lib/python3.8/dist-packages
Requires: requests, tenacity, langsmith, PyYAML, dataclasses-json, pydantic, SQLAlchemy, numpy, async-timeout, langchain-community, jsonpatch, aiohttp, langchain-core
Required-by: 


In [2]:
!wget -q https://www.dropbox.com/s/zoj9rnm7oyeaivb/new_papers.zip
!unzip -q new_papers.zip -d new_papers

# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Files - PDFs
- ChromaDB - with more meta data?
- Source info
- gpt-3.5-turbo API
- HuggingFace Embeddings
- Instuctor Embeddings


## Setting up LangChain


In [3]:
import os
from getpass import getpass

# enter your api key
OPENAI_API_KEY = getpass("Enter your API key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [5]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.8/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


## Load multiple and process documents

In [7]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('./new_papers/new_papers/', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [8]:
len(documents)

142

In [9]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [10]:
len(texts)

659

In [11]:
texts[3]

Document(page_content='3 Using Tools and Act 9\n3.1 Calling another model . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 9\n3.2 Information retrieval . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 11\n3.2.1 Retrieval-augmented language models . . . . . . . . . . . . . . . . . . . . . . . . . . . 11\n3.2.2 Querying search engines . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 12\n3.2.3 Searching and navigating the web . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 12\n3.3 Computing via Symbolic Modules and Code Interpreters . . . . . . . . . . . . . . . . . . . . . 13\n3.4 Acting on the virtual and physical world . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 13\n4 Learning to reason, use tools, and act 15\n4.1 Supervision . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 15', metadata={'source': 'new_papers/new_papers/Augmenting LL

## HF Embeddings

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"

hf = HuggingFaceEmbeddings(model_name=model_name)

## HF Instructor Embeddings

In [13]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

# 70m 24.3s

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [15]:
!ls /root/.cache/torch/sentence_transformers

BAAI_bge-large-en-v1.5
BAAI_bge-small-en-v1.5
hkunlp_instructor-xl
sentence-transformers_all-MiniLM-L6-v2
sentence-transformers_all-mpnet-base-v2
sentence-transformers_clip-ViT-B-32
sentence-transformers_multi-qa-mpnet-base-dot-v1
thenlper_gte-large


In [ ]:
# docker cp c8324b70601d://root/.cache/torch/sentence_transformers/hkunlp_instructor-xl /home/rob/Data3/sentence_transformers
# Successfully copied 4.97GB to /home/rob/Data3/sentence_transformers


## create the DB

In [16]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db2' # I renamed this to db2 to avoid issues with the 'YT_Chroma_DB_Multi_doc_retriever_Langchain_Part1.ipynb' notebook.

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

# 24.4s

In [17]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [18]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## Make a retriever

In [19]:
retriever = vectordb.as_retriever()

In [20]:
docs = retriever.get_relevant_documents("What is Flash attention?")

In [21]:
len(docs)

4

In [22]:
docs[0]

Document(page_content='access.\nWe propose FlashAttention , a new attention algorithm that computes exact attention with far fewer\nmemory accesses. Our main goal is to avoid reading and writing the attention matrix to and from HBM.\nThis requires (i) computing the softmax reduction without access to the whole input (ii) not storing the large\nintermediate attention matrix for the backward pass. We apply two well-established techniques to address\nthese challenges. (i) We restructure the attention computation to split the input into blocks and make several\npasses over input blocks, thus incrementally performing the softmax reduction (also known as tiling). (ii) We\nstore the softmax normalization factor from the forward pass to quickly recompute attention on-chip in the\nbackward pass, which is faster than the standard approach of reading the intermediate attention matrix from\nHBM. We implement FlashAttention in CUDA to achieve ﬁne-grained control over memory access and', metadata={'

In [23]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [24]:
retriever.search_type

'similarity'

In [25]:
retriever.search_kwargs

{'k': 3}

## Make a chain

In [26]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [27]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [28]:
# full example
query = "What is Flash attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 FlashAttention is a new attention algorithm that computes exact attention with fewer memory accesses, using
techniques like tiling and storing the softmax normalization factor. It enables faster and higher quality
Transformers, and is up to 3x faster than the standard attention implementation for common sequence lengths.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [29]:
# break it down
query = "What does IO-aware mean?"
llm_response = qa_chain(query)
process_llm_response(llm_response)
# llm_response

 IO-aware means optimizing for reading and writing to fast/slow memory, which is typically done in computer
science to increase performance.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [30]:
query = "What is tiling in flash-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Tiling is a technique used in FlashAttention to split the input into blocks and make several passes over
input blocks, thus incrementally performing the softmax reduction.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [31]:
query = "What is toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Toolformer is a language model that learns to use external tools such as search engines, calculators, and
translation systems via simple API calls in a self-supervised way.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [32]:
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Toolformer can use search engines, calculators, and translation systems via simple API calls.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [33]:
query = "How many examples do we need to provide for each tool?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 We only need a handful of manually labeled examples for each tool.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [34]:
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 There are two types of retrievers that can be used to augment a LM: dense and sparse. Sparse retrievers work
with sparse bag-of-words representations of the documents and the queries (Robertson and Zaragoza, 2009). In
contrast, dense neural retrievers use a dense query and dense representations of the documents (Borgeaud et
al., 2022; Izacard et al., 2022). Beyond these two types of retrievers, LM can also be augmented with
reasoning strategies (Wei et al., 2022c; Taylor et al., 2022; Yang et al., 2022c inter alia) and external
tools (Press et al., 2022; Gao et al., 2022; Liu et al., 2022b inter alia).


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [35]:
query = "What are the differences between REALM and RAG?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 REALM (Guu et al. ,2020) was the first method to jointly train end-to-end a retrieval system with an encoder
LM, while RAG (Lewis et al. ,2020) jointly fine-tunes the retriever with a sequence-to-sequence model.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/ReACT.pdf


In [36]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity',
 <langchain_community.vectorstores.chroma.Chroma at 0x7fd631017eb0>)

In [37]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


## Deleteing the DB

In [38]:
# !zip -r db.zip ./db
!zip -r db2.zip ./db2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: db2/ (stored 0%)
  adding: db2/chroma-collections.parquet (deflated 50%)
  adding: db2/index/ (stored 0%)
  adding: db2/index/index_847b5d4d-ec97-4f77-8e4b-bf7dc4d2c3f6.bin (deflated 10%)
  adding: db2/index/uuid_to_id_847b5d4d-ec97-4f77-8e4b-bf7dc4d2c3f6.pkl (deflated 39%)
  adding: db2/index/index_metadata_847b5d4d-ec97-4f77-8e4b-bf7dc4d2c3f6.pkl (deflated 14%)
  adding: db2/index/id_to_uuid_847b5d4d-ec97-4f77-8e4b-bf7dc4d2c3f6.pkl (deflated 36%)
  adding: db2/chroma-embeddings.parquet (deflated 19%)


In [39]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
# !rm -rf db/
!rm -rf db2/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Starting again loading the db

restart the runtime

In [40]:
# !unzip db.zip
!unzip db2.zip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Archive:  db2.zip
   creating: db2/
  inflating: db2/chroma-collections.parquet  
   creating: db2/index/
  inflating: db2/index/index_847b5d4d-ec97-4f77-8e4b-bf7dc4d2c3f6.bin  
  inflating: db2/index/uuid_to_id_847b5d4d-ec97-4f77-8e4b-bf7dc4d2c3f6.pkl  
  inflating: db2/index/index_metadata_847b5d4d-ec97-4f77-8e4b-bf7dc4d2c3f6.pkl  
  inflating: db2/index/id_to_uuid_847b5d4d-ec97-4f77-8e4b-bf7dc4d2c3f6.pkl  
  inflating: db2/chroma-embeddings.parquet  


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [42]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [43]:
# persist_directory = 'db'
persist_directory = 'db2'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

In [44]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [45]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [46]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [47]:
# full example
query = "How much money did Pando raise?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

InvalidDimensionException: Dimensionality of (1536) does not match index dimensionality (768)

### Chat prompts

In [48]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [49]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}
